In [33]:
! pip install llama_index

  Obtaining dependency information for llama_index from https://files.pythonhosted.org/packages/8c/1f/7498c09badf50543f56179504aa7e495fc1058a10b8cb64051341d79f88f/llama_index-0.7.15-py3-none-any.whl.metadata
  Using cached llama_index-0.7.15-py3-none-any.whl.metadata (4.5 kB)
Using cached llama_index-0.7.15-py3-none-any.whl (626 kB)
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/usr/local/lib/python3.11/site-packages/llama_index/VERSION'
Consider using the `--user` option or check the permissions.



In [10]:
! pip install 'langchain[all]' 

  Obtaining dependency information for O365<3.0.0,>=2.0.26 from https://files.pythonhosted.org/packages/38/77/dbdfd721e2d0fdf4c1bb4423d1fe88f3cee27bd932a86c92e6c4dc9e8543/O365-2.0.27-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 153.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for anthropic<0.4,>=0.3 from https://files.pythonhosted.org/packages/ab/68/5cab3e37d57f848b0a5ffc2cb5149770b62c1ae304f2f9fbf7c6c5e5521f/anthropic-0.3.6-py3-none-any.whl.metadata
  Obtaining dependency information for arxiv<2.0,>=1.4 from https://files.pythonhosted.org/packages/f0/06/9b9d553d93e25ae27ec5ba794216afb1af248e43d85a35e922a85cbb396a/arxiv-1.4.8-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.8/157.8 kB 251.8 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to buil

In [34]:
from llama_index import SimpleDirectoryReader, GPTListIndex, ServiceContext, GPTVectorStoreIndex, LLMPredictor, PromptHelper
from langchain.chat_models import ChatOpenAI

import sys
import os


# simpleDirectoryReader => use to read our knowledge base
# GPTListIndex => use to index our data
# GPTSimpleVectorIndex => to load our index data 
# LLMPredictor => Large language model predictor to define which llm we are using
# PromptHelper => mainly used for user prompt, what will be the message size, number of token etc

In [42]:
os.environ["OPENAI_API_KEY"] = "<ADD YOUR API KEY>"

In [43]:
def createVectorIndex(path):
    max_input = 4096
    tokens = 256
    chunk_size = 600
    chunk_overlap_ratio = 0.1

    prompt_helper = PromptHelper(max_input, tokens, chunk_overlap_ratio, chunk_size_limit=chunk_size)

    # defining LLM 

    llmPredictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=tokens))


    # load data
    docs = SimpleDirectoryReader(path).load_data()

    # create vector index
    service_context = ServiceContext.from_defaults(llm_predictor=llmPredictor, prompt_helper=prompt_helper)
    vectorIndex = GPTVectorStoreIndex.from_documents(documents=docs, service_context=service_context) 
    vectorIndex.save_to_disk('vectorIndex.json')

    return vectorIndex

In [44]:
vectorIndex = createVectorIndex('Knowledge')

RetryError: RetryError[<Future at 0x128249890 state=finished raised RateLimitError>]

In [ ]:
def answerMe(vectorIndex):
    vIndex = GPTVectorStoreIndex.load_from_disk(vectorIndex)
    while True:
        prompt = input("Please ask: ")
        response = vIndex.query(prompt, response_mode="compact")
        print(f"Response: {response} \n")